In [1]:
## Why do Feature Selection ??

# Reduces Overfitting: Less redundant data means less opportunity to make decisions based on noise.
# Improves Accuracy: Less misleading data means modeling accuracy improves.
# Reduces Training Time: Less data means that algorithms train faster.

In [2]:
# Imports
from sklearn.utils import shuffle
import pandas as pd
import numpy as np
np.random.seed(10)
import sklearn
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

print (sklearn.__version__)
print (pd.__version__)
print (np.__version__)

0.20.0
0.23.4
1.15.2


In [3]:
# this class takes care for scaling the features to the scale of 0-1
# we are doing the scaling with this cap because we use sigmoid activation fxn in logistic which 
# also has the range from 0-1
class Normalizer:

    def __init__(self):
        self.sc = MinMaxScaler()
    
    def scale(self, X, dtype):
        if dtype=='train':
            XX = self.sc.fit_transform(X)
        elif dtype=='test':
            XX = self.sc.transform(X)
        else:
            return None
        return XX

#### METHOD-1
# Univariate Feature Selection

In [4]:


# Select those features that have strong relationship with the output variable
df = pd.read_csv('H:/SELF/Yashu/Files/iris.csv')
df = shuffle(df)
print (df.head(2))
print ('*'*20)
norm = Normalizer()
X = norm.scale(df.iloc[:,:-1].values, 'train')
Y = df.iloc[:,-1].values
print (X[:2], Y[:2])
print ('*'*20)
# feature extraction
test = SelectKBest(score_func=chi2, k=2)
fit = test.fit(X, Y)
# summarize scores
np.set_printoptions(precision=3)
print(fit.scores_)
print ('*'*20)
features = fit.transform(X)
# summarize selected features
print(features[0:2,:])

     Unnamed: 0  Sepal.Length  Sepal.Width  Petal.Length  Petal.Width  \
87           88           6.3          2.3           4.4          1.3   
111         112           6.4          2.7           5.3          1.9   

        Species  
87   versicolor  
111   virginica  
********************
[[0.58389262 0.55555556 0.125      0.57627119 0.5       ]
 [0.74496644 0.58333333 0.29166667 0.72881356 0.75      ]] ['versicolor' 'virginica']
********************
[22.522 11.377  4.471 26.862 30.478]
********************
[[0.576 0.5  ]
 [0.729 0.75 ]]


#### METHOD-2
# Recursive Feature Elimination (Backwards)

In [5]:
# The Recursive Feature Elimination (or RFE) works by recursively removing attributes and
# building a model on those attributes that remain. It uses the model accuracy to identify which attributes 
# (and combination of attributes) contribute the most to predicting the target attribute

df = pd.read_csv('H:/SELF/Yashu/Files/iris.csv')
df = shuffle(df)
print (df.head(2))
print ('*'*20)
norm = Normalizer()
X = norm.scale(df.iloc[:,:-1].values, 'train')
Y = df.iloc[:,-1].values
print (X[:2], Y[:2])
print ('*'*20)
model = LogisticRegression()
rfe = RFE(model, 2)
fit = rfe.fit(X, Y)
print(("Selected Features: %s") % fit.support_)

     Unnamed: 0  Sepal.Length  Sepal.Width  Petal.Length  Petal.Width  \
87           88           6.3          2.3           4.4          1.3   
100         101           6.3          3.3           6.0          2.5   

        Species  
87   versicolor  
100   virginica  
********************
[[0.584 0.556 0.125 0.576 0.5  ]
 [0.671 0.556 0.542 0.847 1.   ]] ['versicolor' 'virginica']
********************
Selected Features: [ True False False False  True]


c:\users\lenovo\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\lenovo\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
c:\users\lenovo\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\lenovo\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", F

#### METHOD-3
# Using Feature importance from any bagging algorithm

In [8]:
df = pd.read_csv('H:/SELF/Yashu/Files/iris.csv')
df = shuffle(df)
print (df.head(2))
print ('*'*20)
norm = Normalizer()
X = norm.scale(df.iloc[:,:-1].values, 'train')
Y = df.iloc[:,-1].values
print (X[:2], Y[:2])
print ('*'*20)
# feature extraction
model = ExtraTreesClassifier()
model.fit(X, Y)
print (model.feature_importances_)

    Unnamed: 0  Sepal.Length  Sepal.Width  Petal.Length  Petal.Width Species
36          37           5.5          3.5           1.3          0.2  setosa
43          44           5.0          3.5           1.6          0.6  setosa
********************
[[0.242 0.333 0.625 0.051 0.042]
 [0.289 0.194 0.625 0.102 0.208]] ['setosa' 'setosa']
********************
[0.326 0.032 0.034 0.2   0.408]


c:\users\lenovo\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


### Feature Selection and Elimination are mutually exclusive techniques for getting correct set of features for
### your ML model

## Few techniques for elimination are:
### Remove Zero/Less Variance columns
### Remove Columns with many missing values
### Remove Highly +ve/-ve co-related features

## Other techniques for feature selection include: 
### PCA Decomposition
### Auto Encoders